## Hyperparameter optimization using hyperopt

In [1]:
from heartnet_v1 import heartnet, reshape_folds
import tables
import ast # for list import
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from keras.losses import binary_crossentropy
from keras import backend as K

Using TensorFlow backend.


In [2]:
foldname='fold1_noFIR'

In [72]:
random_seed=1
batch_size=64
fold_dir='/media/taufiq/Data/heart_sound/feature/potes_1DCNN/balancedCV/folds/'
log_dir= '/media/taufiq/Data/heart_sound/Heart_Sound/codes/logs/'
bn_momentum = 0.99
eps= 1.1e-5
bias=False
l2_reg=0.
l2_reg_dense=0.
kernel_size=5
maxnorm=10000.
dropout_rate=0.5
dropout_rate_dense=0.
padding='valid'
activation_function='relu'
subsam=2
load_path=False
lr=0.0012843784 
lr_decay=0.0001132885
FIR_train=True
num_filt=(8,4)

params = dict()
print "values initialized"

values initialized


In [4]:
feat = tables.open_file(fold_dir + foldname + '.mat')
x_train = feat.root.trainX[:]
y_train = feat.root.trainY[0, :]
x_val = feat.root.valX[:]
y_val = feat.root.valY[0, :]
train_parts = feat.root.train_parts[:]
val_parts = feat.root.val_parts[0, :]

for i in range(0, y_train.shape[0]):
    if y_train[i] == -1:
        y_train[i] = 0  ## Label 0 for normal 1 for abnormal
for i in range(0, y_val.shape[0]):
    if y_val[i] == -1:
        y_val[i] = 0
x_train, y_train, x_val, y_val = reshape_folds(x_train, x_val, y_train, y_val)
print "loading complete"

(78021, 2500, 1)
(78021, 1)
(9834, 2500, 1)
(9834, 1)
loading complete


In [77]:
def objective(args):
    num_dense = args
    print "num_dense {}".format(args)
    model = heartnet(activation_function, bn_momentum, bias, dropout_rate, dropout_rate_dense,
                 eps, kernel_size, l2_reg, l2_reg_dense, load_path, lr, lr_decay, maxnorm,
                 padding, random_seed, subsam, num_filt, num_dense, FIR_train)
    model.fit(x_train, y_train,
          batch_size=256,
          epochs=15,
          shuffle=True,
          verbose=1,
          initial_epoch=0)
    y_pred = model.predict(x_val)
    loss = K.eval(K.mean(K.variable(K.eval(binary_crossentropy(K.variable(y_val),K.variable(y_pred))))))
    params[str(num_dense)]=loss
    print loss
    return loss    

In [ ]:
from hyperopt import hp, Trials, fmin, tpe
trials = Trials()
best = fmin(objective,
    space=hp.randint('x',400),
    algo=tpe.suggest,
    max_evals=20,
    trials=trials)

num_dense 21
Epoch 1/15
78021/78021 [==============================] - 23s 300us/step - loss: 0.3173 - acc: 0.8345
Epoch 2/15
78021/78021 [==============================] - 19s 237us/step - loss: 0.2686 - acc: 0.8657
Epoch 3/15
78021/78021 [==============================] - 19s 238us/step - loss: 0.2505 - acc: 0.8845
Epoch 4/15
78021/78021 [==============================] - 19s 239us/step - loss: 0.2354 - acc: 0.8938
Epoch 5/15
78021/78021 [==============================] - 19s 239us/step - loss: 0.2269 - acc: 0.8988
Epoch 6/15
78021/78021 [==============================] - 18s 237us/step - loss: 0.2195 - acc: 0.9018
Epoch 7/15
78021/78021 [==============================] - 19s 239us/step - loss: 0.2158 - acc: 0.9044
Epoch 8/15
78021/78021 [==============================] - 19s 240us/step - loss: 0.2116 - acc: 0.90640s - loss: 0.21
Epoch 9/15
78021/78021 [==============================] - 19s 244us/step - loss: 0.2081 - acc: 0.9077
Epoch 10/15
78021/78021 [=============================

In [75]:
params

{'170': 0.4816657,
 '272': 0.47328907,
 '344': 0.43311101,
 '37': 0.53506106,
 '371': 0.47320735,
 '400': 0.49441531,
 '459': 8.7024918,
 '80': 0.48589295}